In [ ]:
import numpy as np
import tifffile
import colorsys
import random
from pathlib import Path
import matplotlib.patches as patches
import matplotlib.patheffects as PathEffects
from matplotlib.transforms import Bbox
import xml.etree.ElementTree as ET
import importlib
import os
import sys
import numpy as np
from matplotlib import pyplot as plt
from multiplex2brightfield import converter, convert_from_file, configuration_presets

In [ ]:
def read_and_visualize_multiplex(filepath, target_channel_names, brightness=0.5, level=1):
    """
    Reads a multiplex image and creates an RGB visualization where each channel 
    has a distinct color, matching channels from the file metadata.
    
    Parameters:
    -----------
    filepath : Path or str
        Path to the OME-TIFF file
    target_channel_names : list
        List of channel names to visualize
    brightness : float
        Overall brightness factor
    level : int
        Pyramid level to read (1 is full resolution, lower numbers are higher resolution if available,
        higher numbers are lower resolution)
    """
    random.seed(7)

    # Read the image and metadata
    with tifffile.TiffFile(filepath) as tif:
        # Get metadata for channel names and pixel size information
        ome_metadata = tif.ome_metadata
        metadata = tif.pages[0].tags['ImageDescription'].value

        # Parse the XML metadata
        root = ET.fromstring(metadata)
        ns_uri = root.tag.split('}')[0].strip('{')
        ns = {'ome': ns_uri}

        # Extract channel names from metadata
        channel_elements = root.findall('.//ome:Channel', ns)
        file_channel_names = [channel.get('Name') for channel in channel_elements if channel.get('Name')]

        # Determine the number of available pyramid levels and adjust level index (zero-based)
        n_levels = len(tif.series[0].levels)
        level_index = level - 1
        print(f"Available pyramid levels: {n_levels} (levels 1 to {n_levels})")
        print(f"Requested level: {level}")
        if level_index >= n_levels:
            print(f"Warning: Requested level {level} not available. Using highest available level {n_levels}")
            level_index = n_levels - 1
        elif level_index < 0:
            print(f"Warning: Requested level {level} not available. Using lowest available level 1")
            level_index = 0

        # Compute effective pixel size at the requested pyramid level
        pixels = root.find('.//ome:Pixels', ns)
        if pixels is not None:
            phys_size_x = pixels.get('PhysicalSizeX')
            phys_size_y = pixels.get('PhysicalSizeY')
            phys_size_x_unit = pixels.get('PhysicalSizeXUnit', '')
            phys_size_y_unit = pixels.get('PhysicalSizeYUnit', '')
            try:
                phys_size_x = float(phys_size_x)
                phys_size_y = float(phys_size_y)
            except (ValueError, TypeError):
                phys_size_x = None
                phys_size_y = None

            # Get the shape of the full resolution image (assumed level 1) and the chosen pyramid level.
            base_shape = tif.series[0].levels[0].shape  # full resolution
            current_shape = tif.series[0].levels[level_index].shape

            # Assuming the last two dimensions are (height, width), compute the scaling factors.
            scale_x = base_shape[-1] / current_shape[-1]
            scale_y = base_shape[-2] / current_shape[-2]

            if phys_size_x is not None and phys_size_y is not None:
                effective_phys_size_x = phys_size_x * scale_x
                effective_phys_size_y = phys_size_y * scale_y
                print("Effective pixel size at pyramid level {}:".format(level))
                print("  PhysicalSizeX: {} {}".format(effective_phys_size_x, phys_size_x_unit))
                print("  PhysicalSizeY: {} {}".format(effective_phys_size_y, phys_size_y_unit))
            else:
                print("Pixel size information not found or invalid in metadata.")
        else:
            print("Pixel size information not found in metadata.")

        # Read the requested pyramid level
        img = tif.series[0].levels[level_index].asarray()
        print(f"Image shape at level {level}: {img.shape}")
        if len(img.shape) < 4:
            img = np.expand_dims(img, axis=0)  # Add z-axis if missing.
            print("No z level found. Adding one.")
        print("Channels found in file:", file_channel_names)
        print("Number of channels in data:", img.shape[0])
        print("Number of channels in metadata:", len(file_channel_names))
        
    print("Final image shape:", img.shape)
    
    # Create RGB visualization
    rgb_img_stack = []
    for z in range(img.shape[0]):
        rgb_img = np.zeros((*img.shape[2:], 3), dtype=np.float32)
        base_colors = [
            (255, 0, 0),      # red
            (0, 255, 0),      # green
            (0, 0, 255),      # blue
            (255, 255, 0),    # yellow
            (255, 0, 255),    # magenta
            (0, 255, 255),    # cyan
            (128, 0, 128),    # bop purple
            (255, 165, 0),    # bop orange
            (31, 168, 241),   # bop blue
        ]
        clean_file_names = [name.replace('-', '_').lower() for name in file_channel_names]
        channel_mapping = {name: idx for idx, name in enumerate(clean_file_names)}

        for i, channel_name in enumerate(target_channel_names):
            clean_name = channel_name.replace('-', '_').lower()
            if clean_name in channel_mapping:
                idx = channel_mapping[clean_name]
                color = (np.array(base_colors[i]) / 255.0 if i < len(base_colors) 
                         else np.array(colorsys.hsv_to_rgb(random.uniform(0, 1), 1, 1)))
                channel_data = img[z, idx].astype(float)
                if channel_data.max() > channel_data.min():
                    channel_data = (channel_data - channel_data.min()) / (channel_data.max() - channel_data.min())
                for c in range(3):
                    rgb_img[..., c] += brightness * len(target_channel_names) * channel_data * color[c]

        rgb_img = np.clip(rgb_img, 0, 1)
        rgb_img_stack.append(rgb_img)
    
    if img.shape[0] == 1:
        return rgb_img
    else:
        return np.array(rgb_img_stack)


In [ ]:


def add_scalebar(ax, pixel_size, scalebar_length, length_unit='mm',
                 bar_color='white', text_color='white',
                 bar_thickness_pt=3, margin_pt=10,
                 manual_bar_thickness_pt=None, text_offset_multiplier=1.75,
                 show_text=True, font_size=10):
    """
    Adds a scale bar with fixed screen-space offsets and an optional text label whose vertical
    offset is proportional to the actual bar thickness. If a manual thickness is provided,
    that thickness is used for both the bar and for computing the text offset.
    
    Parameters:
      ax                      : The matplotlib Axes to draw on.
      pixel_size              : Image pixel size in µm.
      scalebar_length         : Desired physical length of the scale bar.
      length_unit             : 'mm' or 'um' (case‐insensitive).
      bar_color               : Color of the scale bar.
      text_color              : Color of the label text.
      bar_thickness_pt        : Base bar thickness in points (will be halved if not overridden).
      margin_pt               : Margin in points from the axes edge (the effective margin is half this value).
      manual_bar_thickness_pt : Optional manual thickness (in points) for the bar.
      text_offset_multiplier  : Factor to multiply the bar thickness to determine the vertical offset
                                for the label (e.g. 1.0 places the text one line thickness above the bar).
      show_text               : Boolean flag; if False, only the scale bar is drawn.
      font_size               : Font size for the scale bar label text.
    """
    # Convert scalebar_length to micrometers and create the label.
    if length_unit.lower() == 'mm':
        length_um = scalebar_length * 1000
        label = f"{scalebar_length} mm"
    elif length_unit.lower() in ['um', 'µm']:
        length_um = scalebar_length
        label = f"{scalebar_length} µm"
    else:
        raise ValueError("Unknown length unit; use 'mm' or 'um'.")
    
    # Get image dimensions (in pixels) from the first image on the axis.
    im = ax.get_images()[0]
    img_data = im.get_array()
    height_px, width_px = img_data.shape[:2]
    
    # Calculate desired scale bar length in image pixels.
    scale_bar_px = length_um / pixel_size  # (µm) / (µm/pixel)
    # Determine fraction of the axis width that this bar occupies.
    bar_width_frac = scale_bar_px / width_px

    # Get the axes' bounding box in display (pixel) coordinates.
    renderer = ax.figure.canvas.get_renderer()
    bbox = ax.get_window_extent(renderer=renderer)
    axis_width_pixels = bbox.width
    axis_height_pixels = bbox.height

    # Convert fixed offsets from points to pixels.
    margin_pixels = margin_pt * ax.figure.dpi / 72.0
    # Move the bar/text halfway toward the corner.
    effective_margin = margin_pixels / 2.0

    # Use the manual thickness if provided; otherwise, use half of bar_thickness_pt.
    if manual_bar_thickness_pt is not None:
        bar_thickness_pixels = manual_bar_thickness_pt * ax.figure.dpi / 72.0
    else:
        bar_thickness_pixels = (bar_thickness_pt / 2) * ax.figure.dpi / 72.0

    # Set the text vertical offset proportional to the bar thickness.
    text_offset_pixels = bar_thickness_pixels * text_offset_multiplier

    # Compute the bar's width in display pixels.
    bar_width_pixels = bar_width_frac * axis_width_pixels

    # Determine the lower-left corner of the bar in display space.
    bar_bl_disp = (bbox.x0 + effective_margin, bbox.y0 + effective_margin)
    # Upper-right corner in display space:
    bar_tr_disp = (bbox.x0 + effective_margin + bar_width_pixels,
                   bbox.y0 + effective_margin + bar_thickness_pixels)
    
    # Convert these display coordinates back to normalized (axes) coordinates.
    bl_axes = ax.transAxes.inverted().transform(bar_bl_disp)
    tr_axes = ax.transAxes.inverted().transform(bar_tr_disp)
    rect_width = tr_axes[0] - bl_axes[0]
    rect_height = tr_axes[1] - bl_axes[1]
    
    # Draw the scale bar.
    rect = patches.Rectangle(bl_axes, rect_width, rect_height,
                             transform=ax.transAxes,
                             color=bar_color,
                             clip_on=False)
    ax.add_patch(rect)
    
    # If requested, add the label text.
    if show_text:
        text_disp = (bbox.x0 + effective_margin + bar_width_pixels / 2,
                     bbox.y0 + effective_margin + bar_thickness_pixels + text_offset_pixels)
        text_axes = ax.transAxes.inverted().transform(text_disp)
        txt = ax.text(text_axes[0], text_axes[1], label,
                      transform=ax.transAxes,
                      color=text_color,
                      ha='center', va='bottom', fontsize=font_size)
        # Add a semi-transparent black shadow behind the text.
        txt.set_path_effects([PathEffects.withStroke(linewidth=3, foreground=(0, 0, 0, 0.5))])




In [ ]:



input_filename = "IMC/20230714_EMMA/20230714_EMMA_20230714-181349-894426_Q010.ome.tif"
output_filename = "output/20230714_EMMA_20230714-181349-894426_Q010_H&E.ome.tif"

config = configuration_presets.GetConfiguration()
# del config["haematoxylin"]
del config["components"]["eosinophilic"]
del config["components"]["epithelial"]
del config["components"]["erythrocytes"]
del config["components"]["marker"]

result1 = convert_from_file(
    input_filename,
    output_pixel_size_x=1,
    output_pixel_size_y=1,
    config = config
)

config = configuration_presets.GetConfiguration()
del config["components"]["haematoxylin"]
# del config["eosinophilic"]
del config["components"]["epithelial"]
del config["components"]["erythrocytes"]
del config["components"]["marker"]

result2 = convert_from_file(
    input_filename,
    output_pixel_size_x=1,
    output_pixel_size_y=1,
    config = config
)

config = configuration_presets.GetConfiguration()
del config["components"]["haematoxylin"]
del config["components"]["eosinophilic"]
# del config["epithelial"]
del config["components"]["erythrocytes"]
del config["components"]["marker"]

result3 = convert_from_file(
    input_filename,
    output_pixel_size_x=1,
    output_pixel_size_y=1,
    config = config
)

config = configuration_presets.GetConfiguration()
del config["components"]["haematoxylin"]
del config["components"]["eosinophilic"]
del config["components"]["epithelial"]
# del config["erythrocytes"]
del config["components"]["marker"]

result4 = convert_from_file(
    input_filename,
    output_pixel_size_x=1,
    output_pixel_size_y=1,
    config = config
)

config = configuration_presets.GetConfiguration()

result5 = convert_from_file(
    input_filename,
    output_pixel_size_x=1,
    output_pixel_size_y=1,
    config = config
)


file_path = Path("IMC/20230714_EMMA/20230714_EMMA_20230714-181349-894426_Q010.ome.tif")
target_channels = ['191Ir_DNA', '193Ir_DNA', '141Pr_aSMA', '143Nd_VIM', '146Nd_Col1A1', '150Nd_FN', '162Dy_CD31', '174Yb_Desmin', 
                '142Nd_CAV1', '148Nd_panCK', '161Dy_AQP1', '164Dy_PDPN', '169Tm_LYVE1', '175Lu_eCadherin', '176Yb_EPCAM', '160Gd_Ter119']
rgb_image = read_and_visualize_multiplex(file_path, target_channels, brightness = 0.8)


In [ ]:
# Create figure with two subplots side by side
fig, (ax1, ax2, ax3, ax4, ax5, ax6) = plt.subplots(1, 6, figsize=(16, 11))

file_path = Path("IMC/20230714_EMMA/20230714_EMMA_20230714-181349-894426_Q010.ome.tif")
target_channels = ['191Ir_DNA', '193Ir_DNA', '141Pr_aSMA', '143Nd_VIM', '146Nd_Col1A1', '150Nd_FN', '162Dy_CD31', '174Yb_Desmin', 
                '142Nd_CAV1', '148Nd_panCK', '161Dy_AQP1', '164Dy_PDPN', '169Tm_LYVE1', '175Lu_eCadherin', '176Yb_EPCAM', '160Gd_Ter119']
rgb_image = read_and_visualize_multiplex(file_path, target_channels, brightness = 0.8)

A_rgb_image_cropped = rgb_image[2000:2000+1412, :, :]
A_result1_cropped = result1[:, :, 2000:2000+1412, :]
A_result2_cropped = result2[:, :, 2000:2000+1412, :]
A_result3_cropped = result3[:, :, 2000:2000+1412, :]
A_result4_cropped = result4[:, :, 2000:2000+1412, :]
A_result5_cropped = result5[:, :, 2000:2000+1412, :]


# Display images
im1 = ax1.imshow(A_rgb_image_cropped)
im2 = ax2.imshow(np.transpose(A_result1_cropped[0], (1, 2, 0)))
im3 = ax3.imshow(np.transpose(A_result2_cropped[0], (1, 2, 0)))
im4 = ax4.imshow(np.transpose(A_result3_cropped[0], (1, 2, 0)))
im5 = ax5.imshow(np.transpose(A_result4_cropped[0], (1, 2, 0)))
im6 = ax6.imshow(np.transpose(A_result5_cropped[0], (1, 2, 0)))


# Set titles if provided
ax1.set_title("IMC", fontsize=18)
ax2.set_title("Blue haematoxylin", fontsize=18)
ax3.set_title("Pink eosin", fontsize=18)
ax4.set_title("Purple eosin", fontsize=18)
ax5.set_title("Red blood cells", fontsize=18)
ax6.set_title("Combined", fontsize=18)

ax1.axis('off')
ax2.axis('off')
ax3.axis('off')
ax4.axis('off')
ax5.axis('off')
ax6.axis('off')

import matplotlib.patches as patches

# Add a black box around each axis
for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
    rect = patches.Rectangle((0, 0), 1, 1, transform=ax.transAxes, fill=False, color='black', linewidth=2)
    ax.add_patch(rect)

fig.canvas.draw()  # Ensure renderer is ready

add_scalebar(ax1, pixel_size=1, scalebar_length=500, length_unit='um',
             bar_color='white', text_color='white', show_text=False, font_size=12)

# Adjust layout to prevent overlap
plt.tight_layout()
plt.savefig('figures/stains.pdf', format='pdf', bbox_inches='tight')
plt.show()


In [ ]:
input_filename = "IMC/20230714_EMMA/20230714_EMMA_20230714-181349-894426_Q003.ome.tif"
output_filename = "output/20230714_EMMA_20230714-181349-894426_Q003_H&E.ome.tif"

options = ['Default', 'Aperio CS2', 'Hamamatsu S360', 'Hamamatsu XR', 'Leica GT450', '3DHistech Pannoramic Scan II', 'CyteFinder']

config = configuration_presets.GetConfiguration('Default')
del config["components"]["marker"]

result_Default = convert_from_file(
    input_filename,
    output_filename = "output/Default.ome.tif",
    output_pixel_size_x=1,
    output_pixel_size_y=1,
    config = config
)

config = configuration_presets.GetConfiguration('Aperio CS2')
del config["components"]["marker"]

result_AperioCS2 = convert_from_file(
    input_filename,
    output_filename = "output/AperioCS2.ome.tif",
    output_pixel_size_x=1,
    output_pixel_size_y=1,
    config = config
)

config = configuration_presets.GetConfiguration('Hamamatsu S360')
del config["components"]["marker"]

result_HamamatsuS360 = convert_from_file(
    input_filename,
    output_filename = "output/HamamatsuS360.ome.tif",
    output_pixel_size_x=1,
    output_pixel_size_y=1,
    config = config
)

config = configuration_presets.GetConfiguration('Hamamatsu XR')
del config["components"]["marker"]

result_HamamatsuXR = convert_from_file(
    input_filename,
    output_filename = "output/HamamatsuXR.ome.tif",
    output_pixel_size_x=1,
    output_pixel_size_y=1,
    config = config
)

config = configuration_presets.GetConfiguration('Leica GT450')
del config["components"]["marker"]

result_LeicaGT450 = convert_from_file(
    input_filename,
    output_filename = "output/LeicaGT450.ome.tif",
    output_pixel_size_x=1,
    output_pixel_size_y=1,
    config = config
)

config = configuration_presets.GetConfiguration('3DHistech Pannoramic Scan II')
del config["components"]["marker"]

result_3DHistechPannoramicScanII = convert_from_file(
    input_filename,
    output_filename = "output/3DHistechPannoramicScanII.ome.tif",
    output_pixel_size_x=1,
    output_pixel_size_y=1,
    config = config
)

config = configuration_presets.GetConfiguration('CyteFinder')
del config["components"]["marker"]

result_CyteFinder = convert_from_file(
    input_filename,
    output_filename = "output/CyteFinder.ome.tif",
    output_pixel_size_x=1,
    output_pixel_size_y=1,
    config = config
)

file_path = Path("IMC/20230714_EMMA/20230714_EMMA_20230714-181349-894426_Q003.ome.tif")
target_channels = ['191Ir_DNA', '193Ir_DNA', '141Pr_aSMA', '143Nd_VIM', '146Nd_Col1A1', '150Nd_FN', 
                '162Dy_CD31', '174Yb_Desmin', '142Nd_CAV1', '148Nd_panCK', '161Dy_AQP1', 
                '164Dy_PDPN', '169Tm_LYVE1', '175Lu_eCadherin', '176Yb_EPCAM', '160Gd_Ter119']


rgb_image = read_and_visualize_multiplex(file_path, target_channels, brightness = 0.8, level=1)

In [ ]:

# Create figure with two subplots side by side
fig, axs = plt.subplots(1, 6, figsize=(16, 11))
ax1, ax2, ax3, ax4, ax5, ax6 = axs.flatten()

B_rgb_image_cropped = rgb_image[:, 900:900+2195, :]
B_result_Default_cropped = result_Default[:, :, :, 900:900+2195]
B_result_AperioCS2_cropped = result_AperioCS2[:, :, :, 900:900+2195]
B_result_HamamatsuS360_cropped = result_HamamatsuS360[:, :, :, 900:900+2195]
B_result_HamamatsuXR_cropped = result_HamamatsuXR[:, :, :, 900:900+2195]
B_result_LeicaGT450_cropped = result_LeicaGT450[:, :, :, 900:900+2195]
B_result_3DHistechPannoramicScanII_cropped = result_3DHistechPannoramicScanII[:, :, :, 900:900+2195]
B_result_CyteFinder_cropped = result_CyteFinder[:, :, :, 900:900+2195]

# Display images
im1 = ax1.imshow(B_rgb_image_cropped)
im2 = ax2.imshow(np.transpose(B_result_AperioCS2_cropped[0], (1, 2, 0)))
im3 = ax3.imshow(np.transpose(B_result_HamamatsuS360_cropped[0], (1, 2, 0)))
im4 = ax4.imshow(np.transpose(B_result_HamamatsuXR_cropped[0], (1, 2, 0)))
im5 = ax5.imshow(np.transpose(B_result_LeicaGT450_cropped[0], (1, 2, 0)))
im6 = ax6.imshow(np.transpose(B_result_3DHistechPannoramicScanII_cropped[0], (1, 2, 0)))

# Set titles if provided
ax1.set_title("IMC", fontsize=18)
ax2.set_title("Aperio CS2", fontsize=18)
ax3.set_title("Hamamatsu S360", fontsize=18)
ax4.set_title("Hamamatsu XR", fontsize=18)
ax5.set_title("Leica GT450", fontsize=18)
ax6.set_title("Pannoramic SCAN II", fontsize=18)

ax1.axis('off')
ax2.axis('off')
ax3.axis('off')
ax4.axis('off')
ax5.axis('off')
ax6.axis('off')

import matplotlib.patches as patches

# Add a black box around each axis
for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
    rect = patches.Rectangle((0, 0), 1, 1, transform=ax.transAxes, fill=False, color='black', linewidth=2)
    ax.add_patch(rect)

fig.canvas.draw()  # Ensure renderer is ready

add_scalebar(ax1, pixel_size=1, scalebar_length=500, length_unit='um',
             bar_color='white', text_color='white', show_text=False, font_size=12)

# Adjust layout to prevent overlap
plt.tight_layout()
plt.savefig('figures/colour_palettes.pdf', format='pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
input_filename = "IMC/20230714_EMMA/20230714_EMMA_20230714-181349-894426_Q011.ome.tif"
output_filename = "output/20230714_EMMA_20230714-181349-894426_Q011_H&E.ome.tif"

config = configuration_presets.GetConfiguration()
del config["components"]["marker"]

config["components"]["haematoxylin"]["intensity"] = 0.5
config["components"]["eosinophilic"]["intensity"] = 1.0
config["components"]["epithelial"]["intensity"] = 1.0
config["components"]["erythrocytes"]["intensity"] = 1.0

result1 = convert_from_file(
    input_filename,
    output_pixel_size_x=1,
    output_pixel_size_y=1,
    config = config
)

config["components"]["haematoxylin"]["intensity"] = 2
config["components"]["eosinophilic"]["intensity"] = 1.0
config["components"]["epithelial"]["intensity"] = 1.0
config["components"]["erythrocytes"]["intensity"] = 1.0

result2 = convert_from_file(
    input_filename,
    output_pixel_size_x=1,
    output_pixel_size_y=1,
    config = config
)

config["components"]["haematoxylin"]["intensity"] = 1.0
config["components"]["eosinophilic"]["intensity"] = 0.5
config["components"]["epithelial"]["intensity"] = 0.5
config["components"]["erythrocytes"]["intensity"] = 0.5

result3 = convert_from_file(
    input_filename,
    output_pixel_size_x=1,
    output_pixel_size_y=1,
    config = config
)

config["components"]["haematoxylin"]["intensity"] = 1.0
config["components"]["eosinophilic"]["intensity"] = 2.0
config["components"]["epithelial"]["intensity"] = 2.0
config["components"]["erythrocytes"]["intensity"] = 2.0

result4 = convert_from_file(
    input_filename,
    output_pixel_size_x=1,
    output_pixel_size_y=1,
    config = config
)

config["components"]["haematoxylin"]["intensity"] = 1.0
config["components"]["eosinophilic"]["intensity"] = 1.0
config["components"]["epithelial"]["intensity"] = 1.0
config["components"]["erythrocytes"]["intensity"] = 1.0

result5 = convert_from_file(
    input_filename,
    output_pixel_size_x=1,
    output_pixel_size_y=1,
    config = config
)


file_path = Path("IMC/20230714_EMMA/20230714_EMMA_20230714-181349-894426_Q011.ome.tif")
target_channels = ['191Ir_DNA', '193Ir_DNA', '141Pr_aSMA', '143Nd_VIM', '146Nd_Col1A1', '150Nd_FN', '162Dy_CD31', '174Yb_Desmin', 
                '142Nd_CAV1', '148Nd_panCK', '161Dy_AQP1', '164Dy_PDPN', '169Tm_LYVE1', '175Lu_eCadherin', '176Yb_EPCAM', '160Gd_Ter119']
rgb_image = read_and_visualize_multiplex(file_path, target_channels, brightness = 0.8)


In [ ]:

# Create figure with two subplots side by side
fig, axs = plt.subplots(1, 6, figsize=(16, 11))
ax1, ax2, ax3, ax4, ax5, ax6 = axs.flatten()

C_rgb_image_cropped = rgb_image[300:300+2514, :, :]
C_result1_cropped = result1[:, :, 500:500+2514, :]
C_result2_cropped = result2[:, :, 500:500+2514, :]
C_result3_cropped = result3[:, :, 500:500+2514, :]
C_result4_cropped = result4[:, :, 500:500+2514, :]
C_result5_cropped = result5[:, :, 500:500+2514, :]

# Display images
im1 = ax1.imshow(C_rgb_image_cropped)
im2 = ax2.imshow(np.transpose(C_result1_cropped[0], (1, 2, 0)))
im3 = ax3.imshow(np.transpose(C_result2_cropped[0], (1, 2, 0)))
im4 = ax4.imshow(np.transpose(C_result3_cropped[0], (1, 2, 0)))
im5 = ax5.imshow(np.transpose(C_result4_cropped[0], (1, 2, 0)))
im6 = ax6.imshow(np.transpose(C_result5_cropped[0], (1, 2, 0)))

# Set titles if provided
ax1.set_title("\nIMC", fontsize=18)

ax2.set_title("$k_{H} = 0.5, k_{E} = 1.0$", fontsize=18)
ax3.set_title("$k_{H} = 2.0, k_{E} = 1.0$", fontsize=18)
ax4.set_title("$k_{H} = 1.0, k_{E} = 0.5$", fontsize=18)
ax5.set_title("$k_{H} = 1.0, k_{E} = 2.0$", fontsize=18)
ax6.set_title("$k_{H} = 1.0, k_{E} = 1.0$", fontsize=18)

ax1.axis('off')
ax2.axis('off')
ax3.axis('off')
ax4.axis('off')
ax5.axis('off')
ax6.axis('off')

# Add a black box around each axis
for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
    rect = patches.Rectangle((0, 0), 1, 1, transform=ax.transAxes, fill=False, color='black', linewidth=2)
    ax.add_patch(rect)

fig.canvas.draw()  # Ensure renderer is ready

add_scalebar(ax1, pixel_size=1, scalebar_length=500, length_unit='um',
             bar_color='white', text_color='white', show_text=False, font_size=12)

# Adjust layout to prevent overlap
plt.tight_layout()
plt.savefig('figures/intensities.pdf', format='pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import pikepdf

def crop_pdf_bottom(input_path, output_path, percentage=10):
    """
    Crop a percentage from the bottom of each PDF page using pikepdf.
    """
    # Validate percentage
    if not 0 < percentage < 100:
        raise ValueError("Percentage must be between 0 and 100")
    
    # Open the PDF
    pdf = pikepdf.Pdf.open(input_path)
    
    # Process each page
    for page in pdf.pages:
        # Get the current page box
        box = page.MediaBox
        
        # Calculate dimensions
        original_height = box[3] - box[1]  # top - bottom
        height_to_remove = float(original_height) * (percentage / 100)
        new_height = float(original_height) - height_to_remove
        
        # Create new MediaBox
        page.MediaBox = [
            float(box[0]),           # left
            float(box[1]),           # bottom
            float(box[2]),           # right
            float(box[1]) + new_height  # new top
        ]
        
        # Also update CropBox if it exists
        if page.get('/CropBox'):
            page.CropBox = page.MediaBox
    
    # Save the modified PDF
    pdf.save(output_path)


input_pdf = 'figures/intensities.pdf'
output_pdf = 'figures/intensities_cropped.pdf'
crop_pdf_bottom(input_pdf, output_pdf, percentage=8.7)

In [ ]:

input_filename = "IMC/20230714_EMMA/20230714_EMMA_20230714-181349-894426_Q017.ome.tif"
output_filename = "output/20230714_EMMA_20230714-181349-894426_Q017_H&E.ome.tiff"

markers = ['CD44', 'aSMA', 'CD31', 'Ter119', 'Ki67']

config = configuration_presets.GetConfiguration('IHC')
config["components"]["marker"]["targets"] = ['CD44']

result1 = convert_from_file(
    input_filename,
    output_filename = "output/CD44.ome.tif",
    output_pixel_size_x=1,
    output_pixel_size_y=1,
    config = config
)

config = configuration_presets.GetConfiguration('IHC')
config["components"]["marker"]["targets"] = ['aSMA']

result2 = convert_from_file(
    input_filename,
    output_filename = "output/aSMA.ome.tif",
    output_pixel_size_x=1,
    output_pixel_size_y=1,
    config = config
)
config = configuration_presets.GetConfiguration('IHC')
config["components"]["marker"]["targets"] = ['CD31']

result3 = convert_from_file(
    input_filename,
    output_filename = "output/CD44.ome.tif",
    output_pixel_size_x=1,
    output_pixel_size_y=1,
    config = config
)
config = configuration_presets.GetConfiguration('IHC')
config["components"]["marker"]["targets"] = ['Ter119']

result4 = convert_from_file(
    input_filename,
    output_filename = "output/CD31.ome.tif",
    output_pixel_size_x=1,
    output_pixel_size_y=1,
    config = config
)
config = configuration_presets.GetConfiguration('IHC')
config["components"]["marker"]["targets"] = ['Ki67']

result5 = convert_from_file(
    input_filename,
    output_filename = "output/Ter119.ome.tif",
    output_pixel_size_x=1,
    output_pixel_size_y=1,
    config = config
)


file_path = Path("IMC/20230714_EMMA/20230714_EMMA_20230714-181349-894426_Q017.ome.tif")
target_channels = ['191Ir_DNA', '193Ir_DNA', '141Pr_aSMA', '143Nd_VIM', '146Nd_Col1A1', '150Nd_FN', '162Dy_CD31', '174Yb_Desmin', 
                '142Nd_CAV1', '148Nd_panCK', '161Dy_AQP1', '164Dy_PDPN', '169Tm_LYVE1', '175Lu_eCadherin', '176Yb_EPCAM', '160Gd_Ter119']
rgb_image = read_and_visualize_multiplex(file_path, target_channels, brightness = 0.8)

In [ ]:

# Create figure with two subplots side by side
fig, axs = plt.subplots(1, 6, figsize=(16, 11))
ax1, ax2, ax3, ax4, ax5, ax6 = axs.flatten()


D_rgb_image_cropped = rgb_image[3561:4561, 0:1000, :]
D_result1_cropped = result1[:, :, 3561:4561, 0:1000]
D_result2_cropped = result2[:, :, 3561:4561, 0:1000]
D_result3_cropped = result3[:, :, 3561:4561, 0:1000]
D_result4_cropped = result4[:, :, 3561:4561, 0:1000]
D_result5_cropped = result5[:, :, 3561:4561, 0:1000]

# Display images
im1 = ax1.imshow(D_rgb_image_cropped)
im2 = ax2.imshow(np.transpose(D_result1_cropped[0], (1, 2, 0)))
im3 = ax3.imshow(np.transpose(D_result2_cropped[0], (1, 2, 0)))
im4 = ax4.imshow(np.transpose(D_result3_cropped[0], (1, 2, 0)))
im5 = ax5.imshow(np.transpose(D_result4_cropped[0], (1, 2, 0)))
im6 = ax6.imshow(np.transpose(D_result5_cropped[0], (1, 2, 0)))

ax1.set_title("IMC", fontsize=18)
ax2.set_title("CD44-IHC", fontsize=18)
ax3.set_title("aSMA-IHC", fontsize=18)
ax4.set_title("CD31-IHC", fontsize=18)
ax5.set_title("Ter119-IHC", fontsize=18)
ax6.set_title("Ki67-IHC", fontsize=18)

ax1.axis('off')
ax2.axis('off')
ax3.axis('off')
ax4.axis('off')
ax5.axis('off')
ax6.axis('off')

# Add a black box around each axis
for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
    rect = patches.Rectangle((0, 0), 1, 1, transform=ax.transAxes, fill=False, color='black', linewidth=2)
    ax.add_patch(rect)

fig.canvas.draw()  # Ensure renderer is ready

add_scalebar(ax1, pixel_size=1, scalebar_length=500, length_unit='um',
             bar_color='white', text_color='white', show_text=False, font_size=12)

# Adjust layout to prevent overlap
plt.tight_layout()
plt.savefig('figures/IHC.pdf', format='pdf', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:

input_filename = "IMC/20230714_EMMA/20230714_EMMA_20230714-181349-894426_Q004.ome.tif"
output_filename = "output/20230714_EMMA_20230714-181349-894426_Q004_H&E.ome.tif"

file_path = Path("IMC/20230714_EMMA/20230714_EMMA_20230714-181349-894426_Q004.ome.tif")
target_channels = ['191Ir_DNA', '193Ir_DNA', '141Pr_aSMA', '143Nd_VIM', '146Nd_Col1A1', '150Nd_FN', '162Dy_CD31', '174Yb_Desmin', 
                '142Nd_CAV1', '148Nd_panCK', '161Dy_AQP1', '164Dy_PDPN', '169Tm_LYVE1', '175Lu_eCadherin', '176Yb_EPCAM', '160Gd_Ter119']
rgb_image = read_and_visualize_multiplex(file_path, target_channels, brightness = 0.8)

config = configuration_presets.GetConfiguration('Masson Trichrome')
result1 = convert_from_file(
    input_filename,
    output_filename = 'output/MassonTrichrome.ome.tif',
    config = config
)

config = configuration_presets.GetConfiguration( 'PAS')
result2 = convert_from_file(
    input_filename,
    output_filename = 'output/PAS.ome.tif',
    config = config
)
config = configuration_presets.GetConfiguration('Jones Silver')
result3 = convert_from_file(
    input_filename,
    output_filename = 'output/JonesSilver.ome.tif',
    config = config
)
config = configuration_presets.GetConfiguration('Toluidine Blue')
result4 = convert_from_file(
    input_filename,
    output_filename = 'output/ToluidineBlue.ome.tif',
    config = config
)
config = configuration_presets.GetConfiguration('H&E ChatGPT')
result5 = convert_from_file(
    input_filename,
    output_filename = 'output/H&E.ome.tif',
    config = config
)


In [ ]:
# Create figure with two subplots side by side
fig, axs = plt.subplots(1, 6, figsize=(16, 11))
ax1, ax2, ax3, ax4, ax5, ax6 = axs.flatten()


E_rgb_image_cropped = rgb_image[200:200+2584, :, :]
E_result1_cropped = result1[:, :, 200:200+2584, :]
E_result2_cropped = result2[:, :, 200:200+2584, :]
E_result3_cropped = result3[:, :, 200:200+2584, :]
E_result4_cropped = result4[:, :, 200:200+2584, :]
E_result5_cropped = result5[:, :, 200:200+2584, :]

# Display images
im1 = ax1.imshow(E_rgb_image_cropped)
im2 = ax2.imshow(np.transpose(E_result1_cropped[0], (1, 2, 0)))
im3 = ax3.imshow(np.transpose(E_result2_cropped[0], (1, 2, 0)))
im4 = ax4.imshow(np.transpose(E_result3_cropped[0], (1, 2, 0)))
im5 = ax5.imshow(np.transpose(E_result4_cropped[0], (1, 2, 0)))
im6 = ax6.imshow(np.transpose(E_result5_cropped[0], (1, 2, 0)))

ax1.set_title("IMC", fontsize=18)
ax2.set_title("Masson Trichrome", fontsize=18)
ax3.set_title('Periodic acid-Schiff', fontsize=18)
ax4.set_title('Jones Silver', fontsize=18)
ax5.set_title('Toluidine Blue', fontsize=18)
ax6.set_title('H&E', fontsize=18)

ax1.axis('off')
ax2.axis('off')
ax3.axis('off')
ax4.axis('off')
ax5.axis('off')
ax6.axis('off')

# Add a black box around each axis
for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
    rect = patches.Rectangle((0, 0), 1, 1, transform=ax.transAxes, fill=False, color='black', linewidth=2)
    ax.add_patch(rect)

fig.canvas.draw()  # Ensure renderer is ready

add_scalebar(ax1, pixel_size=1, scalebar_length=500, length_unit='um',
             bar_color='white', text_color='white', show_text=False, font_size=12)

# Adjust layout to prevent overlap
plt.tight_layout()
plt.savefig('figures/special.pdf', format='pdf', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:

input_pdf = 'figures/special.pdf'
output_pdf = 'figures/special_cropped.pdf'
crop_pdf_bottom(input_pdf, output_pdf, percentage=4)


In [ ]:
import matplotlib.gridspec as gridspec
# --- Legend drawing helper ---
def draw_legend(ax, markers, offset_y):
    ax.set_xlim(0, 2)
    ax.set_ylim(0, vertical_spacing * len(markers))
    ax.set_aspect('equal', adjustable='box')
    for i, (key, val) in enumerate(markers.items()):
        center_y = vertical_spacing * (len(markers) - 0.5 - i) + offset_y
        y_box = center_y - box_size / 2
        color = (val["R"]/255, val["G"]/255, val["B"]/255)
        rect = patches.Rectangle((horizontal_box_offset, y_box), box_size, box_size,
                                 transform=ax.transData,
                                 facecolor=color, edgecolor="none", clip_on=False)
        ax.add_patch(rect)
        ax.text(horizontal_box_offset + box_size + box_to_text_gap, center_y,
                val["targets"], ha='left', va='center', fontsize=text_font_size, clip_on=False)

# --- Legend data ---
horizontal_box_offset = 0.0
box_to_text_gap = 0.1
text_font_size = 11
box_size = 0.1
vertical_spacing = 0.2
legend_offsets = {"MT": 0.5, "PAS": 0.5, "JS": 0.7, "TB": 0.5, "HE": 0.3}

markers_MT = {
    "nuclei": {"R": 30, "G": 114, "B": 201, "targets": "DNA"},
    "muscle": {"R": 220, "G": 67, "B": 51, "targets": "aSMA, VIM, Desmin"},
    "collagen": {"R": 93, "G": 209, "B": 225, "targets": "Col1A1, FN"},
    "erythrocytes": {"R": 229, "G": 128, "B": 56, "targets": "Ter119"}
}

markers_PAS = {
    "nuclei": {"R": 50, "G": 84, "B": 210, "targets": "DNA"},
    "polysaccharides": {"R": 180, "G": 80, "B": 208, "targets": "pro-SPC, Col1A1, FN"},
    "stroma": {"R": 227, "G": 186, "B": 225, "targets": "aSMA, CAV1, VIM"},
    "1": {"R": 255, "G": 255, "B": 255, "targets": "PDPN, Desmin"}
}

markers_JS = {
    "membranes": {"R": 0, "G": 0, "B": 0, "targets": "Col1A1, FN"},
    "stroma": {"R": 133, "G": 227, "B": 200, "targets": "aSMA, VIM"}
}

markers_TB = {
    "nuclei": {"R": 14, "G": 21, "B": 198, "targets": "DNA"},
    "stroma": {"R": 16, "G": 193, "B": 251, "targets": "aSMA, FN, Col1A1"},
    "1": {"R": 255, "G": 255, "B": 255, "targets": "Desmin, VIM"},
    "metachromasia": {"R": 167, "G": 154, "B": 254, "targets": "pro-SPC, PDPN"}
}

markers_HE = {
    "haematoxylin": {"R": 72, "G": 61, "B": 139, "targets": "DNA"},
    "eosinophilic": {"R": 255, "G": 182, "B": 193, "targets": "aSMA, VIM, pro-SPC"},
    "1": {"R": 255, "G": 255, "B": 255, "targets": "Col1A1, FN, CD31"},
    "2": {"R": 255, "G": 255, "B": 255, "targets": "Desmin, LYVE1"},
    "epithelial": {"R": 199, "G": 143, "B": 187, "targets": "CAV1, panCK, AQP1"},
    "3": {"R": 255, "G": 255, "B": 255, "targets": "PDPN, EPCAM"},
    "erythrocytes": {"R": 186, "G": 56, "B": 69, "targets": "Ter119"}
}

# --- Create the full figure ---
fig = plt.figure(figsize=(16, 17))
gs = gridspec.GridSpec(6, 6, figure=fig, height_ratios=[1, 1, 1, 1, 1, 0.6])

# Plot image panels A–E
axs = [fig.add_subplot(gs[i, j]) for i in range(5) for j in range(6)]

panel_A_titles = ["IMC", "Blue haematoxylin", "Pink eosin", "Purple eosin", "Red blood cells", "Combined"]
panel_A_images = [
    A_rgb_image_cropped,
    np.transpose(A_result1_cropped[0], (1, 2, 0)),
    np.transpose(A_result2_cropped[0], (1, 2, 0)),
    np.transpose(A_result3_cropped[0], (1, 2, 0)),
    np.transpose(A_result4_cropped[0], (1, 2, 0)),
    np.transpose(A_result5_cropped[0], (1, 2, 0))
]

panel_B_titles = ["IMC", "Aperio CS2", "Hamamatsu S360", "Hamamatsu XR", "Leica GT450", "Pannoramic SCAN II"]
panel_B_images = [
    B_rgb_image_cropped,
    np.transpose(B_result_AperioCS2_cropped[0], (1, 2, 0)),
    np.transpose(B_result_HamamatsuS360_cropped[0], (1, 2, 0)),
    np.transpose(B_result_HamamatsuXR_cropped[0], (1, 2, 0)),
    np.transpose(B_result_LeicaGT450_cropped[0], (1, 2, 0)),
    np.transpose(B_result_3DHistechPannoramicScanII_cropped[0], (1, 2, 0))
]


ax2.set_title("$k_{H} = 0.5, k_{E} = 1.0$", fontsize=18)
ax3.set_title("$k_{H} = 2.0, k_{E} = 1.0$", fontsize=18)
ax4.set_title("$k_{H} = 1.0, k_{E} = 0.5$", fontsize=18)
ax5.set_title("$k_{H} = 1.0, k_{E} = 2.0$", fontsize=18)
ax6.set_title("$k_{H} = 1.0, k_{E} = 1.0$", fontsize=18)

panel_C_titles = ["IMC",
                  "$k_{H} = 0.5, k_{E} = 1.0$",
                  "$k_{H} = 2.0, k_{E} = 1.0$",
                  "$k_{H} = 1.0, k_{E} = 0.5$",
                  "$k_{H} = 1.0, k_{E} = 2.0$",
                  "$k_{H} = 1.0, k_{E} = 1.0$"]
panel_C_images = [
    C_rgb_image_cropped,
    np.transpose(C_result1_cropped[0], (1, 2, 0)),
    np.transpose(C_result2_cropped[0], (1, 2, 0)),
    np.transpose(C_result3_cropped[0], (1, 2, 0)),
    np.transpose(C_result4_cropped[0], (1, 2, 0)),
    np.transpose(C_result5_cropped[0], (1, 2, 0))
]


panel_D_titles = ["IMC", "CD44-IHC", "aSMA-IHC", "CD31-IHC", "Ter119-IHC", "Ki67-IHC"]
panel_D_images = [
    D_rgb_image_cropped,
    np.transpose(D_result1_cropped[0], (1, 2, 0)),
    np.transpose(D_result2_cropped[0], (1, 2, 0)),
    np.transpose(D_result3_cropped[0], (1, 2, 0)),
    np.transpose(D_result4_cropped[0], (1, 2, 0)),
    np.transpose(D_result5_cropped[0], (1, 2, 0))
]


panel_E_titles = ["IMC", "Masson Trichrome", "Periodic acid-Schiff", "Jones Silver", "Toluidine Blue", "H&E"]
panel_E_images = [
    E_rgb_image_cropped,
    np.transpose(E_result1_cropped[0], (1, 2, 0)),
    np.transpose(E_result2_cropped[0], (1, 2, 0)),
    np.transpose(E_result3_cropped[0], (1, 2, 0)),
    np.transpose(E_result4_cropped[0], (1, 2, 0)),
    np.transpose(E_result5_cropped[0], (1, 2, 0))
]

# Combine all images and titles
all_titles = panel_A_titles + panel_B_titles + panel_C_titles + panel_D_titles + panel_E_titles
all_images = panel_A_images + panel_B_images + panel_C_images + panel_D_images + panel_E_images

# Plot all image panels
for ax, img, title in zip(axs, all_images, all_titles):
    ax.imshow(img)
    ax.set_title(title, fontsize=14)
    ax.axis('off')
    rect = patches.Rectangle((0, 0), 1, 1, transform=ax.transAxes,
                             fill=False, color='black', linewidth=2)
    ax.add_patch(rect)

# Add scalebars
row_starts = [0, 6, 12, 18, 24]
for i in row_starts:
    add_scalebar(axs[i], pixel_size=1, scalebar_length=500, length_unit='um',
                 bar_color='white', text_color='white', show_text=False, font_size=12)

# Add A–E labels to the left of each row, closer to the images
labels = ['A', 'B', 'C', 'D', 'E']
for i, label in enumerate(labels):
    bbox = axs[i * 6].get_position()
    y_center = (bbox.y0 + bbox.y1) / 2
    x_right_of_letter = bbox.x0
    plt.figtext(x_right_of_letter - 0.006, y_center, label,
                fontsize=20, fontweight='bold',
                va='center', ha='right')

# Legend row
legend_axs = [fig.add_subplot(gs[5, j]) for j in range(6)]
for ax in legend_axs:
    ax.axis('off')

draw_legend(legend_axs[1], markers_MT, legend_offsets["MT"])
draw_legend(legend_axs[2], markers_PAS, legend_offsets["PAS"])
draw_legend(legend_axs[3], markers_JS, legend_offsets["JS"])
draw_legend(legend_axs[4], markers_TB, legend_offsets["TB"])
draw_legend(legend_axs[5], markers_HE, legend_offsets["HE"])

# Save figure
plt.subplots_adjust(hspace=0.15, wspace=0.05)
plt.savefig("figures/composite.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
input_pdf = 'figures/composite.pdf'
output_pdf = 'figures/composite_cropped.pdf'
crop_pdf_bottom(input_pdf, output_pdf, percentage=2)